In [1]:
import pandas as pd

In [2]:
#import csv's for analysis
user_df = pd.read_csv("Original_Files/USER_TAKEHOME.csv")
transaction_df = pd.read_csv("Original_Files/TRANSACTION_TAKEHOME.csv")
products_df = pd.read_csv("Original_Files/PRODUCTS_TAKEHOME.csv")

In [3]:
# Print all DataFrames to see columns for each CSV
print(user_df.columns)
print(transaction_df.columns)
print(products_df.columns)

Index(['ID', 'CREATED_DATE', 'BIRTH_DATE', 'STATE', 'LANGUAGE', 'GENDER'], dtype='object')
Index(['RECEIPT_ID', 'PURCHASE_DATE', 'SCAN_DATE', 'STORE_NAME', 'USER_ID',
       'BARCODE', 'FINAL_QUANTITY', 'FINAL_SALE'],
      dtype='object')
Index(['CATEGORY_1', 'CATEGORY_2', 'CATEGORY_3', 'CATEGORY_4', 'MANUFACTURER',
       'BRAND', 'BARCODE'],
      dtype='object')


## Reviewng data for any null values for database import

In [4]:
##Identify any null values for each csv to determine primary/foreign keys or major anomolies
print("User_DF\n"+ str(user_df.isnull().sum()))
print("\nTransaction_DF\n"+ str(transaction_df.isnull().sum()))
print("\nProducts_DF\n"+ str(products_df.isnull().sum())) 
## Due to Products having no columns with no missing values will propose to include index on exported CSV

User_DF
ID                  0
CREATED_DATE        0
BIRTH_DATE       3675
STATE            4812
LANGUAGE        30508
GENDER           5892
dtype: int64

Transaction_DF
RECEIPT_ID           0
PURCHASE_DATE        0
SCAN_DATE            0
STORE_NAME           0
USER_ID              0
BARCODE           5762
FINAL_QUANTITY       0
FINAL_SALE           0
dtype: int64

Products_DF
CATEGORY_1         111
CATEGORY_2        1424
CATEGORY_3       60566
CATEGORY_4      778093
MANUFACTURER    226474
BRAND           226472
BARCODE           4025
dtype: int64


### Review datatypes

In [5]:
print("User_DF\n"+ str(user_df.dtypes))
print("\nTransaction_DF\n"+ str(transaction_df.dtypes))
print("\nProducts_DF\n"+ str(products_df.dtypes))

User_DF
ID              object
CREATED_DATE    object
BIRTH_DATE      object
STATE           object
LANGUAGE        object
GENDER          object
dtype: object

Transaction_DF
RECEIPT_ID         object
PURCHASE_DATE      object
SCAN_DATE          object
STORE_NAME         object
USER_ID            object
BARCODE           float64
FINAL_QUANTITY     object
FINAL_SALE         object
dtype: object

Products_DF
CATEGORY_1       object
CATEGORY_2       object
CATEGORY_3       object
CATEGORY_4       object
MANUFACTURER     object
BRAND            object
BARCODE         float64
dtype: object


### Date conversions to YYYY-MM-DD Format to align all date fields

In [6]:
#review dates and multiple types found (datetime & YYYY-MM-DD)
print(user_df[['CREATED_DATE', 'BIRTH_DATE']].head())
print("\n" + str(transaction_df[['PURCHASE_DATE', 'SCAN_DATE']].head()))

                CREATED_DATE                 BIRTH_DATE
0  2020-06-24 20:17:54.000 Z  2000-08-11 00:00:00.000 Z
1  2021-01-03 19:53:55.000 Z  2001-09-24 04:00:00.000 Z
2  2023-05-31 18:42:18.000 Z  1994-10-28 00:00:00.000 Z
3  2023-12-26 01:46:22.000 Z                        NaN
4  2023-10-28 11:51:50.000 Z  1972-03-19 00:00:00.000 Z

  PURCHASE_DATE                  SCAN_DATE
0    2024-08-21  2024-08-21 14:19:06.539 Z
1    2024-07-20  2024-07-20 09:50:24.206 Z
2    2024-08-18  2024-08-19 15:38:56.813 Z
3    2024-06-18  2024-06-19 11:03:37.468 Z
4    2024-07-04  2024-07-05 15:56:43.549 Z


In [7]:
#Updating user_df & transaction_df object types to datetime for conversion to simplier date format
user_df[['CREATED_DATE', 'BIRTH_DATE']] = user_df[['CREATED_DATE', 'BIRTH_DATE']].astype('datetime64')
transaction_df['SCAN_DATE'] = transaction_df['SCAN_DATE'].astype('datetime64')

#validation of update
user_df[['CREATED_DATE', 'BIRTH_DATE']].dtypes

CREATED_DATE    datetime64[ns]
BIRTH_DATE      datetime64[ns]
dtype: object

In [8]:
#Converting datetime dates to YYYY-MM-DD formatting
## User_Df update (errors='coerce' in case of null value ignores and moves on with the update as we want to keep nulls)
user_df['CREATED_DATE'] = pd.to_datetime(user_df['CREATED_DATE'], format='%Y-%m-%d', errors='coerce')
user_df['CREATED_DATE'] = user_df['CREATED_DATE'].dt.strftime('%Y-%m-%d')
user_df['BIRTH_DATE'] = pd.to_datetime(user_df['BIRTH_DATE'], format='%Y-%m-%d', errors='coerce')
user_df['BIRTH_DATE'] = user_df['BIRTH_DATE'].dt.strftime('%Y-%m-%d')
## transaction_df update
transaction_df['SCAN_DATE'] = pd.to_datetime(transaction_df['SCAN_DATE'], format='%Y-%m-%d', errors='coerce')
transaction_df['SCAN_DATE'] = transaction_df['SCAN_DATE'].dt.strftime('%Y-%m-%d')

#Validation of updates
print(user_df[['CREATED_DATE', 'BIRTH_DATE']].head())
print("\n" + str(transaction_df[['PURCHASE_DATE', 'SCAN_DATE']].head()))

  CREATED_DATE  BIRTH_DATE
0   2020-06-24  2000-08-11
1   2021-01-03  2001-09-24
2   2023-05-31  1994-10-28
3   2023-12-26         NaN
4   2023-10-28  1972-03-19

  PURCHASE_DATE   SCAN_DATE
0    2024-08-21  2024-08-21
1    2024-07-20  2024-07-20
2    2024-08-18  2024-08-19
3    2024-06-18  2024-06-19
4    2024-07-04  2024-07-05


#### Data Reviews

In [9]:
##review final quantity as 'zero' found within dataset
transaction_df['FINAL_QUANTITY'].unique()

array(['1.00', 'zero', '2.00', '3.00', '4.00', '4.55', '2.83', '2.34',
       '0.46', '7.00', '18.00', '12.00', '5.00', '2.17', '0.23', '8.00',
       '1.35', '0.09', '2.58', '1.47', '16.00', '0.62', '1.24', '1.40',
       '0.51', '0.53', '1.69', '6.00', '2.39', '2.60', '10.00', '0.86',
       '1.54', '1.88', '2.93', '1.28', '0.65', '2.89', '1.44', '2.75',
       '1.81', '276.00', '0.87', '2.10', '3.33', '2.54', '2.20', '1.93',
       '1.34', '1.13', '2.19', '0.83', '2.61', '0.28', '1.50', '0.97',
       '0.24', '1.18', '6.22', '1.22', '1.23', '2.57', '1.07', '2.11',
       '0.48', '9.00', '3.11', '1.08', '5.53', '1.89', '0.01', '2.18',
       '1.99', '0.04', '2.25', '1.37', '3.02', '0.35', '0.99', '1.80',
       '3.24', '0.94', '2.04', '3.69', '0.70', '2.52', '2.27'],
      dtype=object)

In [10]:
#update 'zero' to '0' kept same formating when input into sql will update to float value
transaction_df.loc[transaction_df['FINAL_QUANTITY'] == 'zero', 'FINAL_QUANTITY'] = '0'

In [11]:
##validation of update
transaction_df['FINAL_QUANTITY'].unique()

array(['1.00', '0', '2.00', '3.00', '4.00', '4.55', '2.83', '2.34',
       '0.46', '7.00', '18.00', '12.00', '5.00', '2.17', '0.23', '8.00',
       '1.35', '0.09', '2.58', '1.47', '16.00', '0.62', '1.24', '1.40',
       '0.51', '0.53', '1.69', '6.00', '2.39', '2.60', '10.00', '0.86',
       '1.54', '1.88', '2.93', '1.28', '0.65', '2.89', '1.44', '2.75',
       '1.81', '276.00', '0.87', '2.10', '3.33', '2.54', '2.20', '1.93',
       '1.34', '1.13', '2.19', '0.83', '2.61', '0.28', '1.50', '0.97',
       '0.24', '1.18', '6.22', '1.22', '1.23', '2.57', '1.07', '2.11',
       '0.48', '9.00', '3.11', '1.08', '5.53', '1.89', '0.01', '2.18',
       '1.99', '0.04', '2.25', '1.37', '3.02', '0.35', '0.99', '1.80',
       '3.24', '0.94', '2.04', '3.69', '0.70', '2.52', '2.27'],
      dtype=object)

#### Update Hidden valley ranch convert 

In [12]:
products_df['BRAND'] = products_df['BRAND'].replace("Hidden Valley", "Hidden Valley Ranch")
products_df['MANUFACTURER'] = products_df['MANUFACTURER'].replace("CLOROX NATIONAL", "THE CLOROX COMPANY")


#### Update Barcode to remove .0 value

In [13]:
# Remove '.0' from the end of the string in 'BARCODE' column
# Assuming transaction_df is your DataFrame
transaction_df['BARCODE'] = pd.to_numeric(transaction_df['BARCODE'], errors='coerce')  # This will convert to NaN if not a number
transaction_df['BARCODE'] = transaction_df['BARCODE'].astype(str).str.replace(r'\.0$', '', regex=True)
# transaction_df['BARCODE'] = transaction_df['BARCODE'].astype('Int64')  # Converts to nullable Int64 type, which supports NaN
transaction_df['BARCODE'] = transaction_df['BARCODE'].replace('nan', None)

In [14]:
# Remove '.0' from the end of the string in 'BARCODE' column
# Assuming transaction_df is your DataFrame
products_df['BARCODE'] = pd.to_numeric(products_df['BARCODE'], errors='coerce')  # This will convert to NaN if not a number
products_df['BARCODE'] = products_df['BARCODE'].astype(str).str.replace(r'\.0$', '', regex=True)
# transaction_df['BARCODE'] = transaction_df['BARCODE'].astype('Int64')  # Converts to nullable Int64 type, which supports NaN
products_df['BARCODE'] = products_df['BARCODE'].replace('nan', None)

In [15]:
# Updating values to numeric for SQL Import
transaction_df['FINAL_SALE'] = pd.to_numeric(transaction_df['FINAL_SALE'], errors='coerce')

### Save csv's

In [16]:
user_df.to_csv('user_df.csv',index = False)
transaction_df.to_csv('transaction_df.csv',index = True)
products_df.to_csv('products_df.csv',index = True)